# OpenAIを例としたSASによるAPIの実行
WEBページの情報や簡易的なAPIはfilenameステートメントからの利用も可能ですが、[OpenAI社のAPI](https://platform.openai.com/docs/api-reference)を使うにはPOSTメソッドが必要になります。  

Base SASで対応可能なものは以下になるかと思います。

- HTTPプロシジャ 
- DS2プロシージャ

In [ ]:
* API Key ;
%let api_key = OpenAI API key ;

## HTTPプロシージャ
HTTPプロシージャはパラメータの指定がシンプルで比較的扱いやすくなっており、SAS9.4 M4以降で動作します。  
単純化のためhttpのステータス確認はしていませんが、M5以降の自動マクロ変数SYS_PROCHTTP_STATUS_CODEを使うか、レスポンスヘッダーの1行目を利用します。

複雑な入力をしたい場合はfilenameの参照先の外部ファイルを使用することも可能です。

出力はlibnameのjsonエンジンを使用します。デフォルトでも十分に結果の確認が可能です。

In [2]:
filename response temp ;

%let query = hi. ;

proc http url = "https://api.openai.com/v1/chat/completions"
          method = "POST"
          in = %unquote(%bquote('{"model":"gpt-3.5-turbo", "temperature":0, "messages" : [{"role": "user", "content": "&query."}]}')) 
          out = response
          ;
          headers "Content-type" = "application/json"
                  "Accept" = "application/json"
                  "Authorization" = "Bearer &api_key."
                ;
run ;

libname response json fileref=response ;

data work.content ;
  set response.choices_message ;
  query = "&query." ;
run ;

proc print data = work.content ;
run ;

OBS,ordinal_choices,ordinal_message,role,content,query
1,1,1,assistant,Hello! How can I assist you today?,hi.


### mapファイルを利用したjsonの読み込み

jsonファイルの仕様がわかっている場合は、mapファイルを作成しlibnameで指定することで、データセットを自分で制御することができます。  
おおむねxmlのマップファイルと似たようなものになっています。  

以下では前の呼び出し結果について、ひとつのデータセットにまとめて含まれるようにしています。

In [3]:
filename map temp ;

data _null_ ;
    file map ;
    infile cards4 ;
    input ;
    put _infile_ ;
cards4 ;
{
  "DATASETS": [
    {
      "DSNAME": "RBODY",
      "TABLEPATH": "/root",
      "VARIABLES": [
        {
          "NAME": "model",
          "TYPE": "CHARACTER",
          "PATH": "/root/model"
        },
        {
          "NAME": "prompt_tokens",
          "TYPE": "NUMERIC",
          "PATH": "/root/usage/prompt_tokens"
        },
        {
          "NAME": "completion_tokens",
          "TYPE": "NUMERIC",
          "PATH": "/root/usage/completion_tokens"
        },
        {
          "NAME": "total_tokens",
          "TYPE": "NUMERIC",
          "PATH": "/root/usage/total_tokens"
        },
        {
          "NAME": "content",
          "TYPE": "CHARACTER",
          "PATH": "/root/choices/message/content"
        }
      ]
    }
  ]  
}
;;;;
run ;

libname response json fileref=response map = map;

data work.content ;
  set response.rbody ;
  query = "&query." ;
run ;

proc print data = work.content ;
run ;

OBS,model,prompt_tokens,completion_tokens,total_tokens,content,query
1,gpt-3.5-turbo-0613,9,9,18,Hello! How can I assist you today?,hi.


### DS2プロシージャ
DS2プロシージャは利用されているケースは多くありませんが、9.4M3でも動作しデータセットから一連の操作がプロシージャ内で完結します。  
APIについてはhttpパッケージ、出力結果の取得はjsonパッケージを使用します。  

出力されるjsonのキーがわかっているため、キーを指定することで値を取得します。  
他のAPIで別の階層でも同じキーが使用される場合などは条件を追加する必要があります。

In [5]:
data work.query ;
  query = "hi." ;
run ;

proc ds2 ;
  data work.content(overwrite=yes) ;
    declare nvarchar(1024) content ;
      
    method chat(char query) returns char ; 
      declare package http h() ;
      declare package json j() ;
      declare nvarchar(1024) body payload token content ;
      declare int rc status tokenType parseFlags ;
        
      payload = cats('{"model":"gpt-3.5-turbo-0613", "temperature":0, "messages" : [{"role": "user", "content": "', query, '"}]}') ;
      h.createPostMethod('https://api.openai.com/v1/chat/completions') ;
      h.setRequestContentType('application/json') ;
      h.addRequestHeader('Accept', 'application/json') ;
      h.addRequestHeader('Authorization', %bquote(%unquote('Bearer &api_key.'))) ;
      h.setRequestBodyAsString(payload) ;
      h.executeMethod() ;
    
      /* Check status code of the response */
      status = h.getStatusCode() ;
      if status = 200 then do ; 
        h.getResponseBodyAsString(body, rc) ;
          
        rc = j.createParser(body) ;
        do while (rc = 0) ;
          j.getNextToken( rc, token, tokenType, parseFlags) ;
          if (token = 'content') then do ;
            j.getNextToken( rc, token, tokenType, parseFlags) ;
            content=token ;
          end ;
        end ; 
        rc = j.destroyParser() ;
      end ;
      return content ;
    end ;
           
    method run() ;
      set work.query ;
      content= chat(query) ;
    end ;
      
  enddata ; 
  run ; 
quit ;

proc print data = work.content ;
  var query content ;
run ;

OBS,query,content
1,hi.,Hello! How can I assist you today?
